In [1]:
def calc_intersection_area(box1_list, box2_list):
    box1 ={}
    box1['xmin'] = box1_list[0]
    box1['ymin'] = box1_list[1]
    box1['xmax'] = box1_list[2]
    box1['ymax'] = box1_list[3]

    box2 ={}
    box2['xmin'] = box2_list[0]
    box2['ymin'] = box2_list[1]
    box2['xmax'] = box2_list[2]
    box2['ymax'] = box2_list[3]

    #corrdinates of intersection
    int_x_min = max(box1['xmin'], box2['xmin'])
    int_x_max = min(box1['xmax'], box2['xmax'])
    int_y_min = max(box1['ymin'], box2['ymin'])
    int_y_max = min(box1['ymax'], box2['ymax'])

    if((int_x_max<int_x_min) or (int_y_max<int_y_min)):
        return 0.0

    int_area = (int_y_max-int_y_min)*(int_x_max-int_x_min) 
    
    return int_area

def calc_union_area(box1_list, box2_list):
    box1 ={}
    box1['xmin'] = box1_list[0]
    box1['ymin'] = box1_list[1]
    box1['xmax'] = box1_list[2]
    box1['ymax'] = box1_list[3]

    box2 ={}
    box2['xmin'] = box2_list[0]
    box2['ymin'] = box2_list[1]
    box2['xmax'] = box2_list[2]
    box2['ymax'] = box2_list[3]

    #corrdinates of intersection
    int_x_min = max(box1['xmin'], box2['xmin'])
    int_x_max = min(box1['xmax'], box2['xmax'])
    int_y_min = max(box1['ymin'], box2['ymin'])
    int_y_max = min(box1['ymax'], box2['ymax'])

    if((int_x_max<int_x_min) or (int_y_max<int_y_min)):
        return 0.0

    int_area = (int_y_max-int_y_min)*(int_x_max-int_x_min) 
    area_box1 = (box1['xmax'] - box1['xmin']) * (box1['ymax'] - box1['ymin'])
    area_box2 = (box2['xmax'] - box2['xmin']) * (box2['ymax'] - box2['ymin'])
    
    return float(area_box1+area_box2-int_area)

In [2]:
import json
import glob
import numpy as np
from tqdm import tqdm

# Available files

In [3]:
with open("finaldata_gt.json") as f:
    json_gt = json.load(f)
    
completed_list = []
for name in glob.glob('layout_completed/*.json'):
    num = int(name.split("/")[1].split(".")[0])
    completed_list.append(num)
    
gt_list = []
for name in glob.glob('layout_gt/*.json'):
    num = int(name.split("/")[1].split(".")[0])
    gt_list.append(num)
    
missing_list = []
for name in glob.glob('layout_missing/*.json'):
    num = int(name.split("/")[1].split(".")[0])
    missing_list.append(num)

In [4]:
print(len(json_gt))
print(len(completed_list))
print(len(gt_list))
print(len(missing_list))


3596
2811
920
2966


# Calculating for all 

In [5]:
gt_iou_dict = {}
completed_iou_dict = {}
missing_iou_dict = {}


for obj in tqdm(json_gt, total = len(json_gt)):
    id_obj = obj['id']
    if((id_obj in completed_list) & (id_obj in gt_list) & (id_obj in missing_list)):
        
        height = obj['height']
        width = obj['width']
        
        with open('layout_completed/' + str(id_obj) + '.json') as f:
            completed_json = json.load(f)
            completed_obj_dict = {}
            if(len(completed_json['classes']) == 1):
                completed_json['boxes'] = [completed_json['boxes']]
            for box, clas in zip(completed_json['boxes'], completed_json['classes']):
                if clas in completed_obj_dict.keys():
                    completed_obj_dict[clas].append(box)
                else:
                    completed_obj_dict[clas] = []
                    completed_obj_dict[clas].append(box)
            
        with open('layout_gt/' + str(id_obj) + '.json') as f:
            gt_json = json.load(f)
            gt_obj_dict = {}
            if(len(gt_json['classes']) == 1):
                gt_json['boxes'] = [gt_json['boxes']]
            for box, clas in zip(gt_json['boxes'], gt_json['classes']):
                if clas in gt_obj_dict.keys():
                    gt_obj_dict[clas].append(box)
                else:
                    gt_obj_dict[clas] = []
                    gt_obj_dict[clas].append(box)
            
        with open('layout_missing/' + str(id_obj) + '.json') as f:
            missing_json = json.load(f)
            missing_obj_dict = {}
            if(len(missing_json['classes']) == 1):
                missing_json['boxes'] = [missing_json['boxes']]
            for box, clas in zip(missing_json['boxes'], missing_json['classes']):
                if clas in missing_obj_dict.keys():
                    missing_obj_dict[clas].append(box)
                else:
                    missing_obj_dict[clas] = []
                    missing_obj_dict[clas].append(box)
                    
        #check
        gt_clss = set([o['class'] for o in obj['objects']])
        miss_clss = set(list(missing_obj_dict.keys()))
        compl_clss = set(list(completed_obj_dict.keys()))
        
        if(len(gt_clss-compl_clss)>=len(gt_clss-miss_clss)):
            continue
        
        gt_num = 0
        gt_denum = 0
        completed_num = 0
        completed_denum = 0
        missing_num = 0
        missing_denum = 0
        
        
        for gt_obj in obj['objects']:
            box = gt_obj['box']
            box = [box[0]/width, box[1]/height, box[2]/width, box[3]/height]
            clas = gt_obj['class']
            box_area = (box[3]-box[1])*(box[2]-box[0])
            
            #for gt
            if clas in gt_obj_dict.keys():
                best_overlap = 0
                best_pos = None
                
                for coords in gt_obj_dict[clas]:
                    int_area = calc_intersection_area(coords, box)
                    if(int_area>=best_overlap):
                        best_overlap = int_area
                        best_pos = coords
                        
                gt_obj_dict[clas].remove(best_pos)
                if(len(gt_obj_dict[clas])==0):
                    del gt_obj_dict[clas]
                gt_num += calc_intersection_area(best_pos, box)
                gt_denum += calc_union_area(best_pos, box)  
            else:
                gt_denum += box_area
                
            #for completed
            if clas in completed_obj_dict.keys():
                best_overlap = 0
                best_pos = None
                
                for coords in completed_obj_dict[clas]:
                    int_area = calc_intersection_area(coords, box)
                    if(int_area>=best_overlap):
                        best_overlap = int_area
                        best_pos = coords
                        
                completed_obj_dict[clas].remove(best_pos)
                if(len(completed_obj_dict[clas])==0):
                    del completed_obj_dict[clas]
                completed_num += calc_intersection_area(best_pos, box)
                completed_denum += calc_union_area(best_pos, box)  
            else:
                completed_denum += box_area
                
            
            #for missing
            if clas in missing_obj_dict.keys():
                best_overlap = 0
                best_pos = None
                
                for coords in missing_obj_dict[clas]:
                    int_area = calc_intersection_area(coords, box)
                    if(int_area>=best_overlap):
                        best_overlap = int_area
                        best_pos = coords
                        
                missing_obj_dict[clas].remove(best_pos)
                if(len(missing_obj_dict[clas])==0):
                    del missing_obj_dict[clas]
                missing_num += calc_intersection_area(best_pos, box)
                missing_denum += calc_union_area(best_pos, box)  
            else:
                missing_denum += box_area
                
        if(gt_denum>0):
            gt_iou_dict[id_obj] = float(gt_num/gt_denum)
        else:
            gt_iou_dict[id_obj] = 0
            
        if(completed_denum>0):
            completed_iou_dict[id_obj] = float(completed_num/completed_denum)
        else:
            completed_iou_dict[id_obj] = 0
            
        if(missing_denum>0):
            missing_iou_dict[id_obj] = float(missing_num/missing_denum)
        else:
            missing_iou_dict[id_obj] = 0
          

100%|██████████| 3596/3596 [00:00<00:00, 12990.36it/s]


In [7]:
len(gt_iou_dict)

43

In [6]:
print(np.mean(np.array(list(gt_iou_dict.values()))))
print(np.mean(np.array(list(completed_iou_dict.values()))))
print(np.mean(np.array(list(missing_iou_dict.values()))))

0.33970117290496266
0.2835906955093295
0.2674509629495256
